<a href="https://colab.research.google.com/github/matiapa/itba-sia/blob/master/TPE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TPE 1: Métodos de Búsqueda**

## **Datos del trabajo** 

### Alumnos: 
- Apa, Mati (61223) 
- Beade, Gonzalo (61223)
- D'Agostino, Leonardo Agustín (61223)


### Docentes:
- Rodrigo Ramele
- Juliana Gambini 
- Juan Santos 
- Paula Oseroff
- Eugenia Piñeiro
- Santiago Reyes


### Fecha:
- Primer Cuatrimestre 2022

## **Introducción** 

En el siguiente trabajo práctico implementamos estrategias de búsqueda 
(tanto informadas como no informadas) y las comparamos según su desempeño a la hora de resolver de un desafío. El desarrollo del trabajo gira en torno a la resolución del **Juego de los 8 números** (de ahora en más, \' el Juego\') , cuyas reglas se describen a continuación. 


### Reglas del Juego: 
Dada una tabla de 3x3 rellena con casillas numéricas del 1 al 8 y una casilla vacía o blanca, distribuidas de manera azarosa, el Juego termina al ordenar de menor a mayor los contenidos del cuadrado, siendo la casilla blanca la cola de la sucesión. 

Los únicos movimientos permitidos son aquellos que permutan la casilla vacía con una casilla númerica inmediata, en sentido vertical u horizontal. 


## Estado del Juego: 

Decimos que una disposición de casillas dentro de la tabla de 3x3 representa un estado del Juego. Al permutar la casilla blanca con otra, se induce un nuevo estado en el Juego.

## **Desarrollo** 

Si se va a ejecutar código desde el notebook, correr todos los módulos en el orden en el que lo exponemos. 

### **Importación de librerías a utilizar**

Para este trabajo, vamos a utilizar la librería `matplotlib` y algunas de sus sublibrerías para realizar la representación gráfica del juego. 

In [10]:
from matplotlib import pyplot as plt 
from matplotlib import animation as ani
import numpy as np
import copy
from heapq import *
import graphviz

### **Implementación de la API del Juego**


Empezamos creando una interfaz que represente el estado de un juego cualquier, llamada GameState en nuestra implementación. 

La interfaz se desprende de la tarea en inteligencia artificial y provee métodos que permitirían jugar a cualquier juego de un solo jugador en general. 

In [17]:
class GameState():

  """ Devuelve verdadero si el estado es objetivo/ganador"""
  @property
  def isobjective(self):
    raise 'Not implemented exception'

  """
  Devuelve un conjunto de strings representando movimientos posibles
  según las reglas del juego
  """
  @property
  def game_moves(self):
    raise 'Not implemented exception'

  """
  Ejecuta un movimiento en el juego. De ser válido, devuelve un nuevo estado con dicho movimiento ejecutado
  """
  def make_move(self, m: str):
    raise 'Not implemented exception'

  """
  Todo juego debe poder mostrarse en pantalla, aunque sea una representación muy básica
  """
  def __str__(self): 
    return 'Empty Game!'

Cualquier implementación de esta interfaz debe implementar la igualdad y una apropiada función de hash porque el algoritmo de resolución de búsquedas trabaja con conjuntos. La implementación de set en `Python` es con tablas de hashing. 


In [37]:
# TODO: podriamos aprender a hacerla inmutable 
class EightGameState(GameState):

  offsets = {'f':-3 , 'b':3 , 'l':-1 , 'r':1}

  def __init__(self, **kwargs):
    self.table = kwargs.get('table', EightGameState.__new_table()) # TODO: deberia ser una tupla
    self.zero = kwargs.get('target', self.table.index(0))
    self.__swap(self, self.zero, kwargs.get('source', self.zero)) 

  @staticmethod
  def __new_table():
    x = np.arange(9)
    np.random.shuffle(x)
    return x.tolist()  

  @staticmethod 
  def __swap(self, n, z):
      self.table[n], self.table[z] = self.table[z], self.table[n]

  @property 
  def matrix(self): 
    return np.reshape(self.table, (3, 3))

  @property
  def isobjective(self):
    return all(self.table[i] == (i + 1) for i in range(len(self.table) - 2)) 

  def make_move(self, m: str):
    if m not in self.offsets.keys():
      raise 'Invalid move code'

    if (m == 'f' and self.zero < 3) \
      or (m == 'b' and self.zero >= 6) \
      or (m == 'l' and self.zero % 3 == 0 ) \
      or (m == 'r' and self.zero % 3 == 2 ):  
        return None   
    
    new_zero = self.zero + self.offsets[m]
    return EightGameState(table=copy.deepcopy(self.table), source=self.zero, target=new_zero)

  def __hash__(self): 
    return hash(tuple(self.table))

  def __eq__(self, o): 
    return isinstance(o, EightGameState) and o.table == self.table

  @property
  def game_moves(self):
    return {'b', 'f', 'l', 'r'}

  def __str__(self): 
    return str(np.reshape(self.table, (3, 3)))





In [20]:
# Tests para igualdad y hashes, OK 
x = EightGameState() 
print(x)
y = x.make_move('f')
print(y)
x2 = y.make_move('b')
print(x2)
print(x == x2)
print(hash(x))
print(hash(x2))



[[2 7 1]
 [5 0 3]
 [8 4 6]]
[[2 0 1]
 [5 7 3]
 [8 4 6]]
[[2 7 1]
 [5 0 3]
 [8 4 6]]
True
-5505358399972698653
-5505358399972698653


### **Diseño de clases para el agente buscador de soluciones**

Empezamos implementando una clase `Node`. Esta nos permite hacer crecer un árbol de rótulos de estados a medida que lo necesitemos. Los estamos haciendo genérico para todos los tipos de búsqueda, pero podríamos hacer que cada tipo de búsqueda tenga su propio tipo de `Node` según corresponda. 

In [36]:
class Node:

    LAST_ID = 0

    def __init__(self, game_state: GameState, parent, depth):
        self.game_state = game_state
        self.children = [] # TODO: idealmente es un set pero I'll allow it :) 
        self.parent = parent
        self.depth = depth
        self.id = Node.LAST_ID
        Node.LAST_ID += 1

    def add(self, node):
      self.children.append(node)

    @property
    def state(self): 
      return self.game_state

    def __hash__(self): 
      return hash(self.game_state)

    def __eq__(self, other):
      return type(other) is Node and self.game_state == other.game_state



La implementación de los métodos de búsqueda consiste en una clase resolvedora abstracta `Solver`, que implementa un método genérico, ni informado ni no informado. 

Las clases que heredan de `Solver` implementan los distintos algoritmos vistos en clase. Esta implementación muestra que es una familia de algoritmos, y solo cambia la elección del próximo estado a elegir a partir del estado actual en el árbol de estados. 

In [29]:
class Solver():

    '''
      La funcion score es una funcion que 
      - dado un nodo representante de un estado del juego -
      permite  saber su puntaje. Mientras menor sea el puntaje, mejor. 
      El algoritmo selecciona al que menor valor de score tiene. 
      Si se quiere seleccionar al que mayor valor de score tenga, se lo puede multiplicar por -1
    '''
    def score(self, node): 
      raise 'Not implemented exception'

    def __init__(self, game_state: GameState):
      self.initial_state = game_state
      self.root = Node(self.initial_state, None, 0) 

    @property 
    def initial_node(self): 
      return self.root

    def __iter__(self):
      # self.iter_done = False 
      self.frontier = [] # La Frontera es un Heap (AKA Priority Queue). No tiene sentido agregar y ordenar a futuro, mantenelo ordenado
      self.explored = set() 
      heappush(self.frontier, (self.score(self.root), 0, self.root))
      return self

    ''' 
      En cada iteracion devuelve 
        Excepcion si no pudo encontrar solucion
        Un Nodo valido si encontro solucion
        None si sigue buscando 
      El iterador no se destruye al encontrar una solucion, va a seguir buscando y la hoja solucion muere ahi    
    '''
    def __next__(self):

      if len(self.frontier) == 0:  # TODO: aca podriamos aplicar lo de la profundidad en una extension del metodo
        raise StopIteration
      
      n = heappop(self.frontier)[2]
      self.explored.add(n.game_state) # como es un set, no pasa nada si ya estaba  

      if n.state.isobjective: 
        return n  # TODO: devolver todo el camino no solo el final
      
      for m in n.game_state.game_moves: 
        new_state = n.game_state.make_move(m)
        if new_state is not None and new_state not in self.explored:
          node = Node(new_state, n, n.depth+1) 
          n.add(node)
          heappush(self.frontier, (self.score(node), node.id, node)) # Le agrego el ID para romper desempates  # me esta siempre pusheando al heap 
    
      print(n.game_state)
      return None




In [30]:
class SolverBPA(Solver): 
  def score(self, node): 
    return node.depth

In [31]:
class SolverBPP(Solver):  ## SE QUEDA TRABADO ENTRE DOS ESTADOS REPITE!!!
  def score(self, node): 
    return -node.depth

In [35]:
# Este es un ejemplo sencillo que tiene enrocados los ultimos 4 valores 
# En un par de pasos lo encuentra 
# gs = GameState(table=[1, 2, 3, 4, 8, 5, 7, 6, 0], target=5, source=5) 

gs = EightGameState(table=[1, 2, 3, 4, 5, 0, 7, 8, 6], target=5, source=5)

print(gs.isobjective)
solver = SolverBPP(gs)
iterator = iter(solver)

n = None
while n is None:
  n = next(iterator)

print(n.game_state.matrix)
print(n.game_state.isobjective)

Se han truncado las últimas 5000 líneas del flujo de salida.
 [0 5 7]]
[[1 6 8]
 [3 2 4]
 [5 0 7]]
[[1 6 8]
 [3 2 4]
 [5 7 0]]
[[6 8 4]
 [1 2 7]
 [3 5 0]]
[[6 8 4]
 [1 5 2]
 [3 0 7]]
[[0 8 4]
 [6 1 2]
 [3 5 7]]
[[6 8 4]
 [3 1 2]
 [5 7 0]]
[[6 8 4]
 [3 2 7]
 [5 1 0]]
[[6 8 4]
 [3 2 7]
 [5 0 1]]
[[6 8 4]
 [3 2 7]
 [0 5 1]]
[[3 6 8]
 [5 2 4]
 [0 1 7]]
[[3 6 8]
 [5 2 4]
 [1 0 7]]
[[3 6 8]
 [5 2 4]
 [1 7 0]]
[[3 6 8]
 [2 1 4]
 [5 0 7]]
[[3 6 8]
 [2 1 4]
 [0 5 7]]
[[3 6 8]
 [2 1 4]
 [5 7 0]]
[[3 6 8]
 [2 4 7]
 [5 1 0]]
[[3 6 8]
 [2 4 7]
 [5 0 1]]
[[3 6 8]
 [2 4 7]
 [0 5 1]]
[[2 3 6]
 [5 4 8]
 [0 1 7]]
[[2 3 6]
 [5 4 8]
 [1 0 7]]
[[2 3 6]
 [5 4 8]
 [1 7 0]]
[[2 3 6]
 [4 1 8]
 [5 0 7]]
[[2 3 6]
 [4 1 8]
 [0 5 7]]
[[2 3 6]
 [4 1 8]
 [5 7 0]]
[[2 3 6]
 [4 8 7]
 [5 1 0]]
[[2 3 6]
 [4 8 7]
 [5 0 1]]
[[2 3 6]
 [4 8 7]
 [0 5 1]]
[[4 2 3]
 [5 8 6]
 [0 1 7]]
[[4 2 3]
 [5 8 6]
 [1 0 7]]
[[4 2 3]
 [5 8 6]
 [1 7 0]]
[[4 2 3]
 [8 1 6]
 [5 0 7]]
[[4 2 3]
 [8 1 6]
 [0 5 7]]
[[4 2 3]
 [8 1 6]
 [5 7 0]]
[[4 2

### **Clases para visualización del árbol de estados**

In [33]:
def node_label(node): 
  return str(node.game_state.matrix)

def build_graphviz_tree(node, graph):
    graph.node(str(node.id), node_label(node))
    for child in node.children:
        build_graphviz_tree(child, graph)
        graph.edge(str(node.id), str(child.id))

def build_graphviz_branch(node, graph):
    graph.node(str(node.id), node_label(node))

    if node.parent != None:
        build_graphviz_branch(node.parent, graph)
        graph.edge(str(node.parent.id), str(node.id))

def renderTree(root):
    graph = graphviz.Digraph('Decision tree')
    build_graphviz_tree(root, graph)
    graph.render(directory='out', view=True)

def renderBranch(leaf):
    graph = graphviz.Digraph('Solution branch')
    build_graphviz_branch(leaf, graph)
    graph.render(directory='out', view=True)

In [34]:
renderBranch(n)
renderTree(solver.initial_node)


### **Implementación de la visualización del juego**

In [ ]:
## TREMENDO TODO

### **Ejecución y comparación de los métodos de búsqueda**


In [ ]:
## TREMENDO TODO